In [1]:
import numpy as np
import scipy as sp
from skimage import io
from matplotlib import pyplot as plt
from skimage import transform
from tqdm import tqdm
from time import time
import pickle
import os
import sys
import pandas as pd
from glob import glob
import mrcfile
from skimage import registration as im_reg_met
from pathlib import Path
import warnings
import json

warnings.filterwarnings('ignore')
%matplotlib inline

sys.path.append('./compute results/')
from registration_functions import (
    precompute_w_params, run_fbm,
    run_fbm_laguerre, run_fast_fbm_laguerre,
    fixed_image_precompute 
)
from util_functions import normalize, apply_transform, mean_std_normalize, save_arr_mrc

In [7]:
from pathlib import Path

global_path = '/home/ubuntu/Data/cryo-em samples/synthetic_data_2023'
folders = ['small_trans']#, 'no_trans']#['no_trans', 'small_trans', 'big_trans']
models = ['model6185']# 'model2096']#, 'model6185']
names = [('_stack.mrc', '_info.csv')]#, ('_iter_stack.mrc', '_iter_info.csv')]
nums = {models[0]:1}#, models[1]:1}

In [8]:
results_path = Path(global_path) / 'results_new'
os.makedirs(str(results_path), exist_ok=True)

functions = {'fbm': run_fbm, 'fbm_laguerre': run_fbm_laguerre, 'fast_fbm_laguerre': run_fast_fbm_laguerre}



In [9]:
method = 'fbm_laguerre'#'fast_fbm_laguerre'
method_path = results_path / method
os.makedirs(str(method_path), exist_ok=True)

func = functions[method]

In [10]:
image_radius = 120
pixel_sampling = 0.5
com_offset_initial = 10

lag_func_num=50
lag_scale=5
# lag_func_num=50
# lag_scale=5
lag_num_dots = 3000
center = 128, 128
normalization = 'standard'


input_values = {'image_radius': image_radius,
                'pixel_sampling': pixel_sampling,
                'com_offset_initial': com_offset_initial,
                'normalization':normalization}

if 'laguerre' in method:
    input_values['lag_func_num'] = lag_func_num
    input_values['lag_scale'] = lag_scale
    input_values['lag_num_dots'] = lag_num_dots
    

In [11]:
params = precompute_w_params(image_radius=image_radius,
                            pixel_sampling=pixel_sampling, com_offset_initial=com_offset_initial,
                             lag_func_num=lag_func_num, lag_scale=lag_scale, 
                             lag_num_dots=lag_num_dots, compute_zeros=False)
print(params.keys())

for folder in folders:
    for model in models:
        for path in glob(str(Path(global_path) / folder / model / f'{nums[model]}_stack.mrc')):
            print(path)
            with mrcfile.open(path) as mrc:
                seq = mrc.data
            fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
            params = fixed_image_precompute(fixed_image,
                                            params, method='fbm', image_radius=input_values['image_radius']) 
#                                             lag_func_num=input_values['lag_func_num'], 
#                                             lag_scale = input_values['lag_scale'],
#                                             lag_num_dots=input_values['lag_num_dots'])

            method_path = results_path / method
            os.makedirs(str(method_path), exist_ok=True)

            opath = method_path / folder / model
            os.makedirs(opath, exist_ok=True)
            opath = method_path / folder / model / str(path).split('/')[-1]
#             if not os.path.exists(opath):
#                 df, fbm_seq, fbm_seq_shift = func(seq, params)
#                 save_arr_mrc(opath, fbm_seq)
#                 df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

        for snr_value in [0.1, 0.5, 1., 2.]:
            for data_name in glob(str(Path(global_path) / folder / model / 
                                      f'{snr_value}/{nums[model]}_stack.mrc')):
                with mrcfile.open(data_name) as mrc:
                    seq = mrc.data
                df, fbm_seq, fbm_seq_shift = func(seq=seq, func_parameters=params, **input_values)
                opath = method_path / folder / model / str(snr_value)
                os.makedirs(str(opath), exist_ok=True)
                opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
                save_arr_mrc(opath, fbm_seq)
                df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))
                with open( method_path / folder /'params.json', 'w') as file:
                    json.dump(input_values, file)
                print(df.head())


120
s_ang = 753.9822368615503
s_rad = 239.99999999999997
bandwidth = 376.99111843077515
len(Im1) 63 len(Ih1) 21 len(Imm) 378
dict_keys(['integration_intervals', 'alphas', 'laguerre_functions', 'precomputed_c1_coefs', 'precomputed_c2_coefs', 'precomputed_coef_exp'])
/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/small_trans/model6185/1_stack.mrc


100%|███████████████████████████████████████████| 99/99 [08:54<00:00,  5.40s/it]


          x         y         ang       ksi  eta_prime  omega_prime  dft_x  \
0 -0.604896  0.519287   75.163130  2.432201   5.654867     1.233304    0.0   
1 -0.807621 -1.295709   25.507958  4.155010   0.628319     0.366658    0.0   
2  1.254380 -2.012466  363.545093  5.269768   1.570796     6.266519    0.0   
3 -1.372799  0.282117  176.383289  2.938909   0.000000     2.999929    0.0   
4 -0.125089 -2.466547  159.194960  4.661718   0.942478     2.699936    0.0   

   dft_y  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  


100%|███████████████████████████████████████████| 99/99 [17:26<00:00, 10.58s/it]


          x         y         ang       ksi  eta_prime  omega_prime  dft_x  \
0  0.394977 -0.633682   75.163130  5.269768   2.199115     1.233304    0.0   
1 -0.807621 -1.295709   25.507958  4.155010   0.628319     0.366658    0.0   
2 -0.938634  3.625217  363.545093  1.824151   5.654867     6.266519    0.0   
3 -1.444192 -0.146860  176.383289  3.242934   0.314159     2.999929    0.0   
4  1.267873  3.423360  159.194960  1.216100   5.026548     2.699936    0.0   

   dft_y  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  


100%|███████████████████████████████████████████| 99/99 [09:03<00:00,  5.49s/it]


          x         y         ang       ksi  eta_prime  omega_prime  dft_x  \
0  0.604896 -0.519287   77.072944  5.573793   2.513274     1.266637    0.0   
1  1.682726  1.171212   25.507958  0.608050   4.084070     0.366658    0.0   
2  1.750862 -2.261927  363.545093  5.371110   1.570796     6.266519    0.0   
3 -1.444192 -0.146860  176.383289  3.242934   0.314159     2.999929    0.0   
4  1.267873  3.423360  159.194960  1.216100   5.026548     2.699936    0.0   

   dft_y  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  


100%|███████████████████████████████████████████| 99/99 [08:24<00:00,  5.10s/it]


          x         y         ang       ksi  eta_prime  omega_prime  dft_x  \
0  0.604896 -0.519287   75.163130  5.573793   2.513274     1.233304    0.0   
1 -0.807621 -1.295709   25.507958  4.155010   0.628319     0.366658    0.0   
2  1.750862 -2.261927  363.545093  5.371110   1.570796     6.266519    0.0   
3 -1.372799  0.282117  176.383289  2.938909   0.000000     2.999929    0.0   
4  0.149787 -2.953554  159.194960  4.763060   0.942478     2.699936    0.0   

   dft_y  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  


In [7]:
for snr_value in [0.1, 0.5, 1., 2.]:
    print('SNR', snr_value)
    gt_df2 = pd.read_csv('/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/results_model2096_small/fbm_laguerre/' \
                         + f'small_trans/model2096/{snr_value}/2_info.csv')
    print(gt_df2.head(2))

SNR 0.1
   Unnamed: 0         x         y        ang       ksi  eta_prime  \
0           0  1.653695  2.653109  63.704244  1.013417   4.712389   
1           1 -1.682726 -1.171212  36.966844  3.749643   0.942478   

   omega_prime  dft_x  dft_y  
0     1.033309   0.94   4.39  
1     0.566653  -0.03  -2.83  
SNR 0.5
   Unnamed: 0         x         y        ang       ksi  eta_prime  \
0           0  0.606726 -2.343313  63.704244  4.965743   1.256637   
1           1  0.807621  1.295709  36.966844  1.013417   3.769911   

   omega_prime  dft_x  dft_y  
0     1.033309   0.15   0.78  
1     0.566653   2.57   0.35  
SNR 1.0
   Unnamed: 0         x         y        ang       ksi  eta_prime  \
0           0  1.653695  2.653109  63.704244  1.013417   4.712389   
1           1  0.807621  1.295709  36.966844  1.013417   3.769911   

   omega_prime  dft_x  dft_y  
0     1.033309   1.07   5.55  
1     0.566653   2.56   0.27  
SNR 2.0
   Unnamed: 0         x         y        ang       ksi  eta_prime